In [2]:
import numpy
import AGG
import pysam

In [5]:
directory = "./mt_test_eval_shortread/"
tp = "tp.vcf.gz"
fp = "fp.vcf.gz"

In [15]:
def extract_features(filename, label):
    labels = []
    length = []
    HFlist = []
    vcf_file = pysam.VariantFile(filename)
    samplelist = list(vcf_file.header.samples)
    for record in vcf_file:
        ref = record.ref
        alt = record.alts
        sample = samplelist[0]
        hf = record.samples[sample]['HF']
        labels.append(label) # truth
        length.append(numpy.abs(len(ref) - len(alt)))
        HFlist.append(hf)





In [4]:
filename = "mt_filtered.annotated.gfa"
graph = AGG.GraphicalGenome(filename=filename)

In [6]:
anchorlist

['A000022',
 'A000044',
 'A000066',
 'A000088',
 'A000110',
 'A000132',
 'A000154',
 'A000176',
 'A000198',
 'A000220',
 'A000242',
 'A000264',
 'A000286',
 'A000308',
 'A000330',
 'A000352',
 'A000374',
 'A000396',
 'A000418',
 'A000440',
 'A000462',
 'A000484',
 'A000506',
 'A000528',
 'A000550',
 'A000572',
 'A000594',
 'A000616',
 'A000638',
 'A000660',
 'A000682',
 'A000704',
 'A000726',
 'A000748',
 'A000770',
 'A000792',
 'A000814',
 'A000836',
 'A000858',
 'A000880',
 'A000902',
 'A000924',
 'A000946',
 'A000968',
 'A000990',
 'A001012',
 'A001034',
 'A001056',
 'A001078',
 'A001100',
 'A001122',
 'A001144',
 'A001166',
 'A001188',
 'A001210',
 'A001232',
 'A001254',
 'A001276',
 'A001298',
 'A001320',
 'A001342',
 'A001364',
 'A001386',
 'A001408',
 'A001430',
 'A001452',
 'A001474',
 'A001496',
 'A001518',
 'A001540',
 'A001562',
 'A001584',
 'A001606',
 'A001628',
 'A001650',
 'A001672',
 'A001694',
 'A001716',
 'A001738',
 'A001760',
 'A001782',
 'A001804',
 'A001826',
 'A0

In [5]:
anchorlist = sorted(graph.anchor.keys())
pos = 49
sanchor = anchorlist[0]
danchor = anchorlist[1]
sanchor, graph.anchor[sanchor], danchor, graph.anchor[danchor],

('A000022',
 {'pos': 22, 'seq': 'TAACCACTCACGGGAGCTCTC'},
 'A000044',
 {'pos': 44, 'seq': 'ATGCATTTGGTATTTTCGTCT'})

In [9]:
102-66

36

In [10]:
sanchor = "A000286"
for edge in graph.outgoing[sanchor]:
    if len(graph.edges[edge].get('variants', "")) !=0:
        if "NC_012920.1" in graph.edges[edge]['reads']:
            print(graph.edges[edge].get('variants'), graph.edges[edge]['seq'], "reference")
        else:
            print(graph.edges[edge].get('variants'), graph.edges[edge]['seq'])


23=2I1X20= CCCTCCCCCCGCTTCTGGCCACAGC
23=3I1X20= CCCCTCCCCCCGCTTCTGGCCACAGC
16=1I6= CC
22=4I1=1X20= CCCCTCCCCCCCGCTTCTGGCCACAGC
23=4I1X20= CCCCCTCCCCCCGCTTCTGGCCACAGC
24=1I20= CCTCCCCCCGCTTCTGGCCACAGC
23=2I1X24=1I18= CCCTCCCCCCGCTTCTGGCCACAGCACTTAAAACACATCTCTGCCAAA
22=3I1=1X20= CCCTCCCCCCCGCTTCTGGCCACAGC
16=2I6= CCC
23=5I1X20= CCCCCCTCCCCCCGCTTCTGGCCACAGC
23=2I1X37=1D4= CCCTCCCCCCGCTTCTGGCCACAGCACTTAAACACATCTCTGCAAA
22=5I1=1X20= CCCCCTCCCCCCCGCTTCTGGCCACAGC
22= C reference
16=3I6= CCCC


In [25]:
sanchor = "A000044"
print(graph.anchor[sanchor])
for edge in graph.outgoing[sanchor]:
    if len(graph.edges[edge].get('variants', "")) !=0:
        if "NC_012920.1" in graph.edges[edge]['reads']:
            print(graph.edges[edge].get('variants'), graph.edges[edge]['seq'], "reference")
        else:
            print(graph.edges[edge].get('variants'), graph.edges[edge]['seq'])

{'pos': 44, 'seq': 'ATGCATTTGGTATTTTCGTCT'}
22= G reference
21=1I1= GG
21=1D22= GGGGGTATGCACGCGATAGCAT
36=1I8= GGGGGGTATGCACGCGGATAGCAT
